# Easy maps with folium#

Folium builds on the data wrangling strengths of the Python ecosystem and the mapping strengths of the Leaflet.js library. It's essentially a Python wrapper around the excellent Leaflet.js mapping library. Manipulate your data in Python, then visualize it in on a Leaflet map via Folium.

The library has a number of built-in tilesets from OpenStreetMap, MapQuest Open, MapQuest Open Aerial, Mapbox, and Stamen, and supports custom tilesets with Mapbox or Cloudmade API keys. Folium supports both GeoJSON and TopoJSON overlays, as well as the binding of data to those overlays to create choropleth maps with color-brewer color schemes.

In [1]:
import folium
from IPython.display import HTML
import json
import pandas as pd
from shapely.geometry import Polygon
from math import cos, pi

In [2]:
folium.__version__

'0.1.3'

In [3]:
# Utility function to embed maps directly in the notebook
def inline_map(m, width=1200, height=500, input_html=False):
    """
    Embeds the HTML source of the map directly into the IPython notebook.
    
    This method will not work if the map depends on any files (json data). Also this uses
    the HTML5 srcdoc attribute, which may not be supported in all browsers.
    """
    if not input_html:
        m._build_map()
        srcdoc = m.HTML.replace('"', '&quot;')
    else:
        srcdoc = m.replace('"', '&quot;')
    return HTML('<iframe srcdoc="{}" '
                 'style="width: {}px; height: {}px; '
                 'border: none"></iframe>'.format(srcdoc, width, height))

### Displaying a map###

In [4]:
# Initialize a folium map simply by specifying the lat/lon to be centered in Berlin
# Berlin coordinates: lat=52.51 / lon=13.42


Available tiles for map:
- "OpenStreetMap"
- "Mapbox Bright" (Limited levels of zoom for free tiles)
- "Stamen Terrain"
- "Cloudmade" (Must pass API key)
- "Mapbox" (Must pass API key)
- "MapQuest" (Must pass API key)

In [5]:
# Test other tile styles


### Markers###

Folium allows to place markers on the map easily. To find out the lat/lon of a given location, a (forward) geocoder is needed. One can use Google Maps, or the following utility function to enable lat/lon popovers:

In [6]:
# Use function lat_lng_popover() on a map and learn how to extract coordinates of places


In [7]:
# Create a map with markers at the following locations: 
# - Augsburger Str. Ubahn (simple_marker)
# - DSR office
# - Wittenbergplatz
# - A park nearby (circle_marker)
# Experiment with markers' colors, icons, sizes (for circle_marker), etc


### Exercises

1. Using the lat-lon popover above, find out the approximate lat/lon of Rio de Janeiro. Compare the coordinates with other Internet sources. 
2. Display a map centered on Rio de Janeiro and with a zoom level such that the full extension of Brazil is displayed.
3. Display a map centered on Rio de Janeiro at an appropriate zoom to resolve the city and place a marker on the Rodrigo de Freitas Lagoon.

### GeoJSON Overlays###

GeoJSON is an open standard format for encoding collections of simple geographical features along with their non-spatial attributes using JavaScript Object Notation. [Wikipedia GeoJSON](https://en.wikipedia.org/wiki/GeoJSON)

In [8]:
# The geographical features are contained in a geojson file
# Source: https://github.com/m-hoerz/berlin-shapes
geo_path = r'berliner-bezirke.geojson'

# Display the data from the geojson file above on top of a map centered in Berlin. 
# Make sure the zoom level is appropriate to display the whole geojson


#### Binding data to GeoJSON Overlays

Visualizing geometries on a map is interesting, but even more so is to be able to show relations and magnitudes on top of it. Folium allows the binding of Pandas dataframes on GeoJSON geometries

In [9]:
geo_path

'berliner-bezirke.geojson'

In [10]:
# GeoJSON data loads like any normal JSON data
geo_json = open(geo_path).read()
geo_data = json.loads(geo_json)

In [11]:
geo_data.keys()

[u'type', u'features']

In [12]:
geo_data['features'][0].keys()

[u'geometry', u'type', u'id', u'properties']

In [13]:
# The geometry is described by its type (Polygon) and coordinates
Polygon(geo_data['features'][0]['geometry']['coordinates'][0])

In [14]:
# The boroughs' names are stored under properties/Name
geo_data['features'][0]['properties']['Name']

u'Mitte'

In [15]:
# All boroughs' names
[gd['properties']['Name'] for gd in geo_data['features']]

[u'Mitte',
 u'Friedrichshain-Kreuzberg',
 u'Pankow',
 u'Charlottenburg-Wilmersdorf',
 u'Spandau',
 u'Steglitz-Zehlendorf',
 u'Tempelhof-Sch\xf6neberg',
 u'Neuk\xf6lln',
 u'Treptow-K\xf6penick',
 u'Marzahn-Hellersdorf',
 u'Lichtenberg',
 u'Reinickendorf']

In [16]:
pd.DataFrame.from_csv?

In [17]:
# Let's bind demographic data contained in a csv file. 
# The binding is made via the a common data column among the GeoJSON and the csv file

# Load data into dataframe, skip file header on initialization


In [18]:
# Create a new data column: population in thousands for simpler understanding and data viz


In [19]:
# We'll use the appropriate options of the geo_json function to produce the final map
map_osm.geo_json?

Object `map_osm.geo_json` not found.


In [20]:
# Display map binding geo_json and dataframe data -- using columns and key_on parameters
# This produces a choropleth map = Choro (Area/Region) + Pleth (quantity)
# Pay attention to the parameter threshold_scale, and adapt it to your particular data


### Exercises

1. Produce a similar map to the one above, but binding the percentage of foreigners per neighborhood in Berlin.
2. Produce a similar map to the one above, but binding the population per neighborhood in Berlin.
